# Exercício 8 - Prompt Engineering - LLaMA 3 (70B) Groq

### Aluno: Pedro Rodrigues Corrêa
### RA: 243236

## Parameters

In [1]:
MODEL_NAME = "llama3-70b-8192"

## Install and Import

In [2]:
!pip install -q datasets
!pip install -q torch
!pip install -q groq
!pip install -q trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.5 MB/s eta 0:00:00


In [3]:
from collections import Counter
from datasets import load_dataset
import re
import torch
import os
from transformers import LlamaTokenizerFast
import pandas as pd
import getpass

## Groq

In [4]:
grok_key = getpass.getpass("Groq api key:")

Groq api key:··········


In [5]:
from groq import Groq

client = Groq(api_key=grok_key)

def groq_chat(content):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": content,
            }
        ],
        model="llama3-70b-8192",
    )
    return chat_completion.choices[0].message.content

## Dataset

In [6]:
# Load the IMDb movie reviews dataset
train_dataset = load_dataset('imdb', split='train')
test_dataset = load_dataset('imdb', split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
# Selecionando os índices propostos pelo exercício
indexes = list(range(8797, 9260)) + list(range(21297, 21760))
test_dataset = test_dataset.select(indexes)

In [8]:
train_labels = train_dataset['label']
item_counts = Counter(train_labels)
print(f'Train Dataset Label breakdown: {item_counts}')

tst_labels = test_dataset['label']
item_counts = Counter(tst_labels)
print(f'Test Dataset Label breakdown: {item_counts}')

Train Dataset Label breakdown: Counter({0: 12500, 1: 12500})
Test Dataset Label breakdown: Counter({0: 463, 1: 463})


## Data Preparation

In [9]:
template_zero = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'.
Sentence:{text}.
Answer:"""

In [10]:
template_few = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'.

Example 1:
Sentence: Incredible acting and stunning visuals. Truly loved it!
Answer:'positive'

Example 2:
Sentence: Boring plot and predictable ending. Not impressed.
Answer:'negative'

Example 3:
Sentence: Best movie this year, totally heartwarming and exciting!
Answer:'positive'

Example 4:
Sentence: Terrible pacing and dull characters. Waste of time.
Answer:'negative'

Now it's your turn:
Sentence:{text}.
Answer:"""

In [11]:
template_few2 = """Your task is to classify sentences' sentiment as 'positive' or 'negative'. Your answer should be one word, either 'positive' or 'negative'. Here's how you should think about it:
- First, consider the overall emotion conveyed by the sentence. Words like 'love', 'happy', 'enjoy' indicate a positive sentiment, while words like 'hate', 'sad', 'disappoint' suggest a negative sentiment.
- Consider the context of the sentence to ensure the sentiment isn't negated by words like 'not' or 'never'.
- If the sentence contains primarily positive terms and no significant negation, it is positive. If it contains negative terms or is negated, it is negative.
Sentence:{text}.
Answer:"""


In [12]:
POSITIVE_LABEL = "positive"
NEGATIVE_LABEL = "negative"

def preprocess(example):
    pattern = re.compile(r'\s+<.*?>')

    text  = example["text"]
    label = example["label"]

    text = re.sub(pattern, ' ', text)
    return {
        "text": text.strip(),
        "class": POSITIVE_LABEL if label == 1 else NEGATIVE_LABEL
    }

def mapper(data, template):
    test_dataset = data.map(lambda example: {'text': example['text'].replace("<br />", " ")})
    test_dataset = test_dataset.map(lambda example: {"text": template.format(**example)})
    test_dataset = test_dataset.map(lambda example: {"class": POSITIVE_LABEL if example["label"] == 1 else NEGATIVE_LABEL})

    return test_dataset

In [14]:
test_zero = mapper(test_dataset, template_zero)
test_few = mapper(test_dataset, template_few)
test_few2 = mapper(test_dataset, template_few2)

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

Map:   0%|          | 0/926 [00:00<?, ? examples/s]

## Validação

In [22]:
val_zero = [test_zero[0].get('class'), test_zero[-1].get('class')]
val_few = [test_few[0].get('class'), test_few[-1].get('class')]
val_few2 = [test_few2[0].get('class'), test_few2[-1].get('class')]

val_zero, val_few, val_few2

(['negative', 'positive'], ['negative', 'positive'], ['negative', 'positive'])

In [25]:
output_val_zero =[groq_chat(test_zero['text'][0]), groq_chat(test_zero['text'][-1])]
output_val_few =[groq_chat(test_few['text'][0]), groq_chat(test_few['text'][-1])]
output_val_few2 =[groq_chat(test_few2['text'][0]), groq_chat(test_few2['text'][-1])]

output_val_zero, output_val_few, output_val_few2

(['Negative', 'positive'],
 ["'negative'", 'positive'],
 ['Negative', 'positive'])

## Avaliação

In [ ]:
from tqdm import tqdm

def eval_model(data):
    predictions = []

    labels = data["class"]
    reviews = data["text"]

    for review in tqdm(reviews):
        predicted = groq_chat(review).lower()
        predictions.append(predicted)

    correct = sum([1 for p, r in zip(predictions, labels) if p.lower() == r.lower()])
    total = len(predictions)
    acc = correct/total

    print(f'Model Accuracy = {acc*100}%')

  #return predictions_raw, predictions

In [ ]:
# Avaliação Zero-Shot

print("Avaliando Modelo Zero-Shot")
eval_model(test_zero)

Avaliando Modelo Zero-Shot


100%|██████████| 926/926 [53:06<00:00,  3.44s/it]

Model Accuracy = 94.9244060475162%


In [ ]:
# Avaliação Few-Shot

print("Avaliando Modelo Few-Shot")
eval_model(test_few)

Avaliando Modelo Few-Shot


100%|██████████| 926/926 [1:17:39<00:00,  5.03s/it]

Model Accuracy = 90.38876889848812%


In [ ]:
# Avaliação Few-Shot Guiado

print("Avaliando Modelo Few-Shot Guiado")
eval_model(test_few2)

Avaliando Modelo Chain-Of-Thought


100%|██████████| 926/926 [57:52<00:00,  3.75s/it]

Model Accuracy = 95.03239740820734%


# Referências

Fábio Grassiotto